# Retrieval Augmented Generation Experiment Example

## Installations

In [1]:
# !pip install --quiet --force-reinstall prompttools

## Setup imports and API keys

We'll import the relevant `prompttools` modules to setup our experiment.

In [2]:
from prompttools.experiment import ChromaDBExperiment, OpenAICompletionExperiment, OpenAIChatExperiment
from prompttools.harness import RetrievalAugmentedGenerationExperimentationHarness

We will be using OpenAI's LLM in this example. You can set up your API key here.

In [3]:
import os

os.environ["OPENAI_API_KEY"] = ""  # Put your key here

### Setup data for ChromaDB Experiment

There are two main steps in Retrieval Augmented Generation. We will start with the retrieval step.

First, we will setup a vector database experiment. We will insert documents the DB with different embedding functions (vectorizer), and query the results.

For this example, we will use ChromaDB, but you use other vector databases as well.

For detailed explanation about each step, have a look at the ChromaDB example notebook.

In [4]:
import chromadb
from chromadb.utils import embedding_functions


emb_fns = [
    embedding_functions.SentenceTransformerEmbeddingFunction(model_name="paraphrase-MiniLM-L3-v2"),
    embedding_functions.DefaultEmbeddingFunction(),
]  # default is "all-MiniLM-L6-v2"
emb_fns_names = ["paraphrase-MiniLM-L3-v2", "default"]

chroma_client = chromadb.Client()
# You can also create and use `chromadb.PersistentClient` or `chromadb.HttpClient`
TEST_COLLECTION_NAME = "TEMPORARY_COLLECTION"
try:
    chroma_client.delete_collection(TEST_COLLECTION_NAME)
except Exception:
    pass
collection_name = TEST_COLLECTION_NAME

use_existing_collection = False  # Specify that we want to create a collection during the experiment

# Documents that will be added into the database
add_to_collection_params = {
    "documents": ["This is a document", "This is another document", "This is the document."],
    "metadatas": [{"source": "my_source"}, {"source": "my_source"}, {"source": "my_source"}],
    "ids": ["id1", "id2", "id3"],
}

# Our test queries
query_collection_params = {"query_texts": ["This is a another query document"]} #["This is a query document", "This is a another query document"]}


# Set up the experiment
vdb_experiment = ChromaDBExperiment(
    chroma_client,
    collection_name,
    use_existing_collection,
    query_collection_params,
    emb_fns,
    emb_fns_names,
    add_to_collection_params,
)

/Users/kevin/miniconda3/envs/prompttools/lib/python3.11/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
/Users/kevin/miniconda3/envs/prompttools/lib/python3.11/site-packages/torch/utils/tensorboard/__init__.py:6: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  ) < LooseVersion("1.15"):
/Users/kevin/miniconda3/envs/prompttools/lib/python3.11/site-packages/tensorflow/python/debug/cli/debugger_cli_common.py:19: DeprecationWarning: module 'sre_constants' is deprecated
  import sre_constants


You can visualize the results and see what documents have been fetched.

In [5]:
vdb_experiment.visualize()

/Users/kevin/miniconda3/envs/prompttools/lib/python3.11/site-packages/chromadb/utils/read_write_lock.py:29: DeprecationWarning: notifyAll() is deprecated, use notify_all() instead
  self._read_ready.notifyAll()


,embed_fn,top doc ids,distances,documents,latency
0,paraphrase-MiniLM-L3-v2,"[id2, id3, id1]","[13.375584602355957, 16.815608978271484, 16.913410186767578]","[This is another document, This is the document., This is a document]",0.008317
1,default,"[id2, id1, id3]","[0.7673601508140564, 0.8709302544593811, 0.9072309732437134]","[This is another document, This is a document, This is the document.]",0.022273


### Setup up Retrieval Augmented Generation Experiment

After setting up your vector database experiment, we can set up the LLM experiment that will consume the documents retrieved from the vector DB. We need:

1. LLM experiment (we will use `OpenAICompletionExperiment` here, but you can use something else as well)
2. LLM arguments (this will be passed into the LLM experiment)
3. A function to extract documents from the resuls of the vector DB experiment

These are the arguments we will use for our LLM experiment `OpenAICompletionExperiment`. For an example with `OpenAIChatExperiment` (that uses `gpt-3.5-turbo`, scroll further below).

In [6]:
models = ["babbage-002"]  # If you want to use "gpt-3,5-turbo", look further below for an example
prompts = ["What happened on 01/01/2025?", "Who is the 50th president?"]
temperatures = [0.0, 1.0]
# You can add more parameters that you'd like to test here.

llm_arguments = {"model": models, "prompt": prompts, "temperature": temperatures}

This function extracts the list of documents from each row of the vector DB experiment result.

In [7]:
def _extract_doc_from_row(row: 'pandas.core.series.Series'):
    return row['documents']

We pass in everything into the RAG experiment

In [8]:
rag_experiment = RetrievalAugmentedGenerationExperimentationHarness(
    vector_db_experiment = vdb_experiment,
    llm_experiment_cls = OpenAICompletionExperiment,
    llm_arguments = llm_arguments,
    extract_document_fn = _extract_doc_from_row,
)

In [9]:
rag_experiment.run()
rag_experiment.visualize()

/Users/kevin/miniconda3/envs/prompttools/lib/python3.11/site-packages/chromadb/utils/read_write_lock.py:29: DeprecationWarning: notifyAll() is deprecated, use notify_all() instead
  self._read_ready.notifyAll()


,prompt,temperature,response,latency
0,Given these documents:\nThis is another document\nThis is the document.\nThis is a document\n\nWhat happened on 01/01/2025?,0.0,\nThis is a document\nThis is a document\nThis is a document\n,0.319791
1,Given these documents:\nThis is another document\nThis is the document.\nThis is a document\n\nWhat happened on 01/01/2025?,1.0,"\n\nReferences in the Notes on this topic contain only a text, not the entire",0.195658
2,Given these documents:\nThis is another document\nThis is the document.\nThis is a document\n\nWho is the 50th president?,0.0,\nA. George Washington\nB. Abraham Lincoln\nC. John F.,0.204074
3,Given these documents:\nThis is another document\nThis is the document.\nThis is a document\n\nWho is the 50th president?,1.0,There are 31 documents.\n 25\nSimon is a 67-year-old,0.193998
4,Given these documents:\nThis is another document\nThis is a document\nThis is the document.\n\nWhat happened on 01/01/2025?,0.0,"The first document is the same as the second document, but the second document is",0.216036
5,Given these documents:\nThis is another document\nThis is a document\nThis is the document.\n\nWhat happened on 01/01/2025?,1.0,Let's STO(1) see.\n```\nAssetWare::STO,0.201545
6,Given these documents:\nThis is another document\nThis is a document\nThis is the document.\n\nWho is the 50th president?,0.0,(A) The 50th president is the 50th president is the,0.202507
7,Given these documents:\nThis is another document\nThis is a document\nThis is the document.\n\nWho is the 50th president?,1.0,"The documents say 50th is President George, the document that George Bush",0.196210


You can also do the same thing with a LLM chat model, such as GPT-3.5-turbo.

In [10]:
chat_models = ["gpt-3.5-turbo"]
messages = [
    [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who was the 50th president?"},
    ],
    [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What happened on 01/01/2025?"},
    ]
]
temperatures = [0.0, 1.0]
# You can add more parameters that you'd like to test here.

llm_chat_arguments = {"model": chat_models, "messages": messages, "temperature": temperatures}

In [11]:
rag_experiment_2 = RetrievalAugmentedGenerationExperimentationHarness(
    vector_db_experiment = vdb_experiment,
    llm_experiment_cls = OpenAIChatExperiment,
    llm_arguments = llm_chat_arguments,
    extract_document_fn = _extract_doc_from_row,
)

rag_experiment_2.run()
rag_experiment_2.visualize()

/Users/kevin/miniconda3/envs/prompttools/lib/python3.11/site-packages/chromadb/utils/read_write_lock.py:29: DeprecationWarning: notifyAll() is deprecated, use notify_all() instead
  self._read_ready.notifyAll()


,temperature,messages,response,latency
0,0.0,"[{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'Given these documents: This is another document This is a document This is the document. Given these documents: This is another document This is the document. This is a document Who was the 50th president?'}]","As of my knowledge, there have been 46 presidents of the United States. Therefore, there is no 50th president.",1.649850
1,1.0,"[{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'Given these documents: This is another document This is a document This is the document. Given these documents: This is another document This is the document. This is a document Who was the 50th president?'}]","As of my knowledge, there have only been 46 presidents of the United States, with the current president being Joe Biden. Therefore, there has not been a 50th president yet.",1.842281
2,0.0,"[{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'Given these documents: This is another document This is a document This is the document. Given these documents: This is another document This is the document. This is a document What happened on 01/01/2025?'}]","Based on the given information, there is no mention of any events or activities that occurred on 01/01/2025.",1.428515
3,1.0,"[{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'Given these documents: This is another document This is a document This is the document. Given these documents: This is another document This is the document. This is a document What happened on 01/01/2025?'}]","Based on the given information, we do not have any specific information about what happened on 01/01/2025. It is not mentioned in the provided documents.",1.734258
4,0.0,"[{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'Given these documents: This is another document This is a document This is the document. Given these documents: This is another document This is the document. This is a document Who was the 50th president?'}]","As of my knowledge, there have been 46 presidents of the United States. Therefore, there is no 50th president.",1.443996
5,1.0,"[{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'Given these documents: This is another document This is a document This is the document. Given these documents: This is another document This is the document. This is a document Who was the 50th president?'}]",There is currently no 50th president as there have only been 46 presidents in the United States as of October 2021.,1.351932
6,0.0,"[{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'Given these documents: This is another document This is a document This is the document. Given these documents: This is another document This is the document. This is a document What happened on 01/01/2025?'}]","Based on the given documents, there is no information about what happened on 01/01/2025.",1.207891
7,1.0,"[{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'Given these documents: This is another document This is a document This is the document. Given these documents: This is another document This is the document. This is a document What happened on 01/01/2025?'}]","Based on the given information, there is no mention of any events or activities that occurred on January 1, 2025.",1.534629
